In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
drive_PATH = '../content/drive/MyDrive/Colab Notebooks/l101.experiments.1'

In [ ]:
!pip install jsonlines datasets git+https://github.com/google-research/bleurt.git evaluate sacrebleu

In [ ]:
import tqdm
import jsonlines
import sys
import numpy as np

# Eval - BLEURT
from datasets import load_metric
bleurt = load_metric("bleurt", module_type="metric")
# Eval - chrf(++)
import evaluate
chrf = evaluate.load("chrf") # (word_order=2) for chrf++

In [ ]:
t05e001_PATH = drive_PATH + '/data-hyperparameter-selection/n128-t0.5-e0.01.jsonl'
t10e001_PATH = drive_PATH + '/data-hyperparameter-selection/n128-t1-e0.01.jsonl'
t20e001_PATH = drive_PATH + '/data-hyperparameter-selection/n128-t2.0-e0.01.jsonl'
t05e002_PATH = drive_PATH + '/data-hyperparameter-selection/n128-t0.5-e0.02.jsonl'
t10e002_PATH = drive_PATH + '/data-hyperparameter-selection/n128-t1-e0.02.jsonl'
t20e002_PATH = drive_PATH + '/data-hyperparameter-selection/n128-t2.0-e0.02.jsonl'

t05e001 = []
t10e001 = []
t20e001 = []
t05e002 = []
t10e002 = []
t20e002 = []

# epsilon = 0.01
with jsonlines.open(t05e001_PATH) as f:
    for line in f.iter():
        t05e001.append(line)
with jsonlines.open(t10e001_PATH) as f:
    for line in f.iter():
        t10e001.append(line)
with jsonlines.open(t20e001_PATH) as f:
    for line in f.iter():
        t20e001.append(line)

# epsilon = 0.02
with jsonlines.open(t05e002_PATH) as f:
    for line in f.iter():
        t05e002.append(line)
with jsonlines.open(t10e002_PATH) as f:
    for line in f.iter():
        t10e002.append(line)
with jsonlines.open(t20e002_PATH) as f:
    for line in f.iter():
        t20e002.append(line)

In [ ]:
def BLEURT_mbrd(candidates):
  k_samples = len(candidates)
  score_matrix = np.zeros((k_samples, k_samples))

  for j1 in range(k_samples):
    references=[candidates[j1][0] for j2 in range(k_samples)]
    predictions=[candidates[j2][0] for j2 in range(k_samples)]
    scores = bleurt.compute(predictions=predictions, references=references)['scores']
    score_matrix[ : ,j1] = scores

  # remove votes for self
  for j1 in range(k_samples):
    for j2 in range(k_samples):
      if j1 == j2:
        score_matrix[j2,j1] = 0

  sum_scores = np.sum(score_matrix, axis=1)
  index = np.argmax(sum_scores)
  mbr_winner = candidates[index]

  return mbr_winner, index, sum_scores, score_matrix

# def CHARF_mbrd(candidates):
#   k_samples = len(candidates)
#   score_matrix = np.zeros((k_samples, k_samples))

#   for j1 in range(k_samples):
#     for j2 in range(k_samples):
#       references=[[candidates[j1][0]]]
#       predictions=[candidates[j2][0]]
#       scores = chrf.compute(predictions=predictions, references=references, word_order=2)['score']
#       score_matrix[j2,j1] = scores

#   # remove votes for self
#   for j1 in range(k_samples):
#     for j2 in range(k_samples):
#       if j1 == j2:
#         score_matrix[j2,j1] = 0

#   sum_scores = np.sum(score_matrix, axis=1)
#   index = np.argmax(sum_scores)
#   mbr_winner = candidates[index]

#   return mbr_winner, index, sum_scores, score_matrix


In [ ]:
num_candidates = 128
num_examples = 200

datasets = {'t05e001' : t05e001,
            't10e001' : t10e001,
            't20e001' : t20e001,
            't05e002' : t05e002,
            't10e002' : t10e002,
            't20e002' : t20e002}

In [ ]:
dataset_name = 't20e001'
dataset = datasets[dataset_name]
for j in tqdm.tqdm(range(num_examples)):
    predictions=[dataset[j]['candidates'][i][0] for i in range(num_candidates)]
    references=[dataset[j]['ref'] for i in range(num_candidates)]
    # dataset[j]['BLEURT_cand_scores'] = np.array(bleurt.compute(predictions=predictions, references=references)['scores'], dtype=float).tolist()
    # MBR BLEURT
    mbr_winner, index, sum_scores, score_matrix = BLEURT_mbrd(dataset[j]['candidates'])
    dataset[j]['MBR_BLEURT_winner'] = mbr_winner
    dataset[j]['MBR_BLEURT_winner_index'] = int(index)
    dataset[j]['MBR_BLEURT_cand_scores'] = sum_scores.tolist()
    dataset[j]['MBR_BLEURT_score_matrix'] = score_matrix.tolist()

bleurt_dataset_PATH = drive_PATH+ f'/data-hyperparameter-selection/bleurt-{dataset_name}.jsonl'
with jsonlines.open(bleurt_dataset_PATH, mode='w') as writer:
    for item in dataset:
        writer.write(item)

In [ ]:
dataset_name = 't20e002'
dataset = datasets[dataset_name]
for j in tqdm.tqdm(range(num_examples)):
    predictions=[dataset[j]['candidates'][i][0] for i in range(num_candidates)]
    references=[dataset[j]['ref'] for i in range(num_candidates)]
    # dataset[j]['BLEURT_cand_scores'] = np.array(bleurt.compute(predictions=predictions, references=references)['scores'], dtype=float).tolist()
    # MBR BLEURT
    mbr_winner, index, sum_scores, score_matrix = BLEURT_mbrd(dataset[j]['candidates'])
    dataset[j]['MBR_BLEURT_winner'] = mbr_winner
    dataset[j]['MBR_BLEURT_winner_index'] = int(index)
    dataset[j]['MBR_BLEURT_cand_scores'] = sum_scores.tolist()
    dataset[j]['MBR_BLEURT_score_matrix'] = score_matrix.tolist()

bleurt_dataset_PATH = drive_PATH+ f'/data-hyperparameter-selection/bleurt-{dataset_name}.jsonl'
with jsonlines.open(bleurt_dataset_PATH, mode='w') as writer:
    for item in dataset:
        writer.write(item)